# 02 - 基础操作

## 学习目标

- 掌握创建 DataFrame 的多种方式
- 学会读取 CSV / Parquet / JSON 数据
- 掌握选择、过滤、排序等基础操作
- 学会列操作：添加列、重命名、删除列
- 理解去重操作

In [ ]:
import daft
from daft import col

## 1. 创建 DataFrame

### 1.1 从 Python 字典创建

In [ ]:
# 从字典创建
df_dict = daft.from_pydict({
    "product_id": ["P001", "P002", "P003"],
    "name": ["手机", "电脑", "耳机"],
    "price": [2999.0, 4999.0, 199.0],
})
df_dict.show()

### 1.2 从 Pandas DataFrame 创建

In [ ]:
import pandas as pd

pdf = pd.DataFrame({
    "product_id": ["P004", "P005"],
    "name": ["平板", "相机"],
    "price": [3499.0, 6999.0],
})

df_pandas = daft.from_pandas(pdf)
df_pandas.show()

## 2. 读取数据

Daft 支持多种数据格式。以下对比三种格式的读取。

In [ ]:
# CSV
df_csv = daft.read_csv("../data/products.csv")
print("CSV Schema:")
print(df_csv.schema())
df_csv.show(3)

In [ ]:
# Parquet（推荐，保留类型信息且体积更小）
df_parquet = daft.read_parquet("../data/products.parquet")
print("Parquet Schema:")
print(df_parquet.schema())
df_parquet.show(3)

In [ ]:
# JSON
df_json = daft.read_json("../data/products.json")
print("JSON Schema:")
print(df_json.schema())
df_json.show(3)

## 3. 选择和过滤

### 3.1 选择列 - `select()`

In [ ]:
# 选择指定列
df = daft.read_parquet("../data/products.parquet")
df_selected = df.select("product_id", "name", "price", "category")
df_selected.show(5)

### 3.2 过滤行 - `where()`

In [ ]:
# 按价格过滤：仅设下限
df_expensive = df.where(col("price") >= 3000)
print("价格 >= 3000 的产品:")
df_expensive.show(5)

In [ ]:
# 按价格过滤：设上下限
df_mid = df.where((col("price") >= 500) & (col("price") <= 2000))
print("价格在 500-2000 之间的产品:")
df_mid.show(5)

In [ ]:
# 按类别过滤
df_electronics = df.where(col("category") == "电子产品")
print("电子产品:")
df_electronics.show(5)

In [ ]:
# 组合条件过滤（直接使用 Daft API）
df_combo = df.where(
    (col("category") == "电子产品") & (col("price") > 1000) & (col("rating").not_null())
)
print("电子产品 + 价格>1000 + 有评分:")
df_combo.show(5)

## 4. 列操作

### 4.1 添加新列 - `with_column()`

In [ ]:
# 添加折扣金额列
df_with_discount = df.with_column("discount_amount", col("original_price") - col("price"))
df_with_discount.select("name", "price", "original_price", "discount_amount").show(5)

### 4.2 重命名列 - `select()` + `alias()`

Daft 0.7.2 中使用 `select(col("old").alias("new"))` 来重命名列。

In [ ]:
# 重命名列
df_renamed = df.select(
    col("product_id").alias("id"),
    col("name").alias("product_name"),
    col("price"),
)
df_renamed.show(5)

### 4.3 删除列 - `exclude()`

In [ ]:
# 删除 description 列
df_no_desc = df.exclude("description")
print("删除 description 后的列:")
print(df_no_desc.column_names)

## 5. 排序和去重

### 5.1 排序 - `sort()`

In [ ]:
# 按价格升序
df_asc = df.sort("price")
print("价格升序（最便宜的 5 个）:")
df_asc.select("name", "price").show(5)

In [ ]:
# 按价格降序
df_desc = df.sort("price", desc=True)
print("价格降序（最贵的 5 个）:")
df_desc.select("name", "price").show(5)

### 5.2 去重 - `distinct()`

In [ ]:
# 去重前后行数对比
count_before = len(df.collect())
df_dedup = df.distinct()
count_after = len(df_dedup.collect())

print(f"去重前: {count_before} 行")
print(f"去重后: {count_after} 行")
print(f"移除了 {count_before - count_after} 条重复记录")

## 6. SQL 查询

Daft 也支持直接使用 SQL 语法查询 DataFrame，对熟悉 SQL 的用户非常友好。

In [ ]:
# 用 SQL 实现与 Python API 等价的查询
df_sql = daft.sql("""
    SELECT name, category, price
    FROM df
    WHERE category = '电子产品' AND price > 1000
    ORDER BY price DESC
    LIMIT 5
""", df=df)

df_sql.show()

## 总结

本节学习了 Daft DataFrame 的基础操作：

| 操作 | 方法 | 示例 |
|------|------|------|
| 创建 | `from_pydict()`, `from_pandas()` | `daft.from_pydict({"a": [1,2]})` |
| 读取 | `read_csv()`, `read_parquet()`, `read_json()` | `daft.read_parquet("data.parquet")` |
| 选择列 | `select()` | `df.select("col1", "col2")` |
| 过滤 | `where()` | `df.where(col("price") > 100)` |
| 添加列 | `with_column()` | `df.with_column("new", expr)` |
| 重命名 | `select()` + `alias()` | `df.select(col("old").alias("new"))` |
| 删除列 | `exclude()` | `df.exclude("col")` |
| 排序 | `sort()` | `df.sort("col", desc=True)` |
| 去重 | `distinct()` | `df.distinct()` |

## 练习题

1. 读取产品数据，找出评分 >= 4.5 且库存 > 0 的产品
2. 添加一个 `price_per_review` 列（price / review_count），并按该列降序排列
3. 选择 `name`, `category`, `price` 三列，将 `name` 重命名为 `product_name`

## 下一步

继续学习 [03_data_processing.ipynb](./03_data_processing.ipynb) —— 掌握聚合、连接和缺失值处理。